In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report,confusion_matrix, plot_confusion_matrix
from statsmodels.stats.proportion import proportions_ztest,confint_proportions_2indep

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
import scipy
import statsmodels

In [4]:
from sklearn.cluster import DBSCAN
import gower

In [5]:
raw = pd.read_csv("sentencing_fips_pop2010 2.csv")

In [6]:
def drop_error_value(df):
    df_temp = df.copy()
    df_temp = df_temp[~((df_temp['spyrs']>=99)|(df_temp['life']=='Y'))]
    df_temp['sp_cj_total_days'] = (df_temp['sp_total_days'])+(df_temp['cj_total_days'])
    df_clean = df_temp[~((df_temp['sp_cj_total_days']<=7) & (df_temp['totpts']>44))]
    
    return df_clean

In [7]:
def sp_county_crime_type(county, crime_type, df):
    match_ind = ''
    if crime_type == 'drug':     ### Expendable
        match_ind = '893.*'
    elif crime_type =='rubbory':
        match_ind = '812.*'
    df = df[df['statut'].str.match(match_ind)== True]
    
    df =df[df['county']== county]
    
    return df
    

In [8]:
def feat_prep(df):
    df_temp = df.copy()
    df_temp['gender_ind']=[1 if x=='MALE' else 0 for x in df_temp['gender']]
    df_temp = pd.concat([df_temp,pd.get_dummies(df_temp['clfely'], prefix='clfely')],axis=1)
    df_temp['sent_year']=df_temp["sentdte"].str[:4]
    df_temp['lsviol']=[0 if x==0 else 1 for x in df_temp['lsviol']]
    df_temp['rpviol']=[0 if x==0 else 1 for x in df_temp['rpviol']]
    df_temp['faviol']=[0 if x==0 else 1 for x in df_temp['faviol']]
    
    return df_temp

In [9]:
def feat_prep(df):
    df_temp = df.copy()
    df_temp['gender_ind']=[1 if x=='MALE' else 0 for x in df_temp['gender']]
    df_temp = pd.concat([df_temp,pd.get_dummies(df_temp['clfely'], prefix='clfely')],axis=1)
    df_temp['sent_year']=df_temp["sentdte"].str[:4]
    df_temp['lsviol']=[0 if x==0 else 1 for x in df_temp['lsviol']]
    df_temp['rpviol']=[0 if x==0 else 1 for x in df_temp['rpviol']]
    df_temp['faviol']=[0 if x==0 else 1 for x in df_temp['faviol']]
    
    return df_temp

In [10]:
import numpy as np
from scipy.stats import ttest_ind
from scipy.stats import t
import pandas as pd

def welch_ttest(x1, x2,alternative):
    
    n1 = x1.size
    n2 = x2.size
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    
    v1 = np.var(x1, ddof=1)
    v2 = np.var(x2, ddof=1)
    
    pooled_se = np.sqrt(v1 / n1 + v2 / n2)
    delta = m1-m2
    
    tstat = delta /  pooled_se
    df = (v1 / n1 + v2 / n2)**2 / (v1**2 / (n1**2 * (n1 - 1)) + v2**2 / (n2**2 * (n2 - 1)))
    
    # two side t-test
    p = 2 * t.cdf(-abs(tstat), df)
    
    # upper and lower bounds
    lb = delta - t.ppf(0.975,df)*pooled_se 
    ub = delta + t.ppf(0.975,df)*pooled_se
  
    return pd.DataFrame(np.array([tstat,df,p,delta,lb,ub]).reshape(1,-1),
                         columns=['T statistic','df','pvalue 2 sided','Difference in mean','lb','ub'])

In [12]:
#!pip3 install gower

  Running setup.py bdist_wheel for gower ... done
  Stored in directory: /Users/hao/Library/Caches/pip/wheels/c0/09/9b/072d54d6ced0f43a179852e3f09532d0131e25ff7cb4e5ee75
Successfully built gower
You are using pip version 10.0.1, however version 22.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
raw_clean = drop_error_value(raw)

# Pinellas county + Drug

In [7]:
df_drug =sp_county_crime_type(county='pinellas',crime_type='drug',df= raw_clean)

In [8]:
df_drug_model = feat_prep(df_drug)

In [9]:
df_drug_model.head()

,Unnamed: 0,fstnm,lstnm,age,dob,race,gender,county,totpts,judge,...,state,pop_2010,sp_cj_total_days,gender_ind,clfely_1ST DEGREE,clfely_1ST/LIFE,clfely_2ND DEGREE,clfely_3RD DEGREE,clfely_LIFE,sent_year
7,7,DOMINIQUE,GAMBLE,19,1988-04-01,BLACK,MALE,pinellas,65.4,LUCE,...,FL,916542.0,1825,1,0,0,1,0,0,2007
53,53,JAMES,PANTLE,30,1977-11-18,WHITE,MALE,pinellas,111.8,"BULONE, J.",...,FL,916542.0,1095,1,1,0,0,0,0,2008
93,93,JOSEPH,JOHNSON,41,1972-10-06,WHITE,MALE,pinellas,113.6,FEDERICO,...,FL,916542.0,900,1,0,0,1,0,0,2014
127,127,RONNIE,YOUNG,20,1989-10-05,BLACK,MALE,pinellas,71.5,LEY,...,FL,916542.0,364,1,0,0,1,0,0,2010
154,154,DONJI,FULLER,34,1974-08-30,BLACK,MALE,pinellas,124.0,NEWTON,...,FL,916542.0,5475,1,1,0,0,0,0,2009


In [13]:
X_continous = df_drug_model.loc[:,['totpts','age']]

In [16]:
X_cont_stand =StandardScaler().fit_transform(X_continous)

In [20]:
df_drug_model['totpts_stand'] = X_cont_stand[:,0]
df_drug_model['age_stand'] = X_cont_stand[:,1]

In [21]:
X = df_drug_model.loc[:,['totpts_stand','age_stand','gender']]

In [28]:
import gower
distance_matrix = gower.gower_matrix(X)

In [25]:
distance_matrix

array([[0.        , 0.10245616, 0.16616678, ..., 0.14810888, 0.01407349,
        0.02624166],
       [0.10245616, 0.        , 0.06371062, ..., 0.12627126, 0.08838267,
        0.09881337],
       [0.16616678, 0.06371062, 0.        , ..., 0.06568809, 0.15209328,
        0.16252398],
       ...,
       [0.14810888, 0.12627126, 0.06568809, ..., 0.        , 0.16218238,
        0.17435054],
       [0.01407349, 0.08838267, 0.15209328, ..., 0.16218238, 0.        ,
        0.01216817],
       [0.02624166, 0.09881337, 0.16252398, ..., 0.17435054, 0.01216817,
        0.        ]], dtype=float32)

In [29]:
from sklearn.cluster import DBSCAN

# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.3, 
                        min_samples=2, 
                        metric="precomputed")

# Fitting the clustering algorithm
dbscan_cluster.fit(distance_matrix)

# Adding the results to a new column in the dataframe
#df_drug_model["cluster"] = dbscan_cluster.labels

AttributeError: 'DBSCAN' object has no attribute 'labels'

In [30]:
df_drug_model["cluster"] = dbscan_cluster.labels_

In [32]:
df_drug_model["cluster"].value_counts()

0    30131
1    12136
Name: cluster, dtype: int64

In [53]:
treat_1 = df_drug_model[(df_drug_model["cluster"]==1)&(df_drug_model["race"]=='BLACK')].sample(2000)
treat_0 = df_drug_model[(df_drug_model["cluster"]==0)&(df_drug_model["race"]=='BLACK')].sample(2000)


In [54]:
treat= treat_0.append(treat_1)

In [55]:
cont_1 = df_drug_model[(df_drug_model["cluster"]==1)&(df_drug_model["race"]=='WHITE')].sample(2000)
cont_0 = df_drug_model[(df_drug_model["cluster"]==0)&(df_drug_model["race"]=='WHITE')].sample(2000)

cont= cont_0.append(cont_1)

In [41]:
import scipy

In [42]:
scipy.stats.ttest_ind(treat.sp_total_days, cont.sp_total_days, equal_var=False)

Ttest_indResult(statistic=9.763856585869128, pvalue=2.240495943817181e-22)

In [43]:
scipy.stats.ttest_ind(treat.sp_total_days, cont.sp_total_days, equal_var=True)

Ttest_indResult(statistic=9.763856585869126, pvalue=2.145683940928613e-22)

In [44]:
import statsmodels

In [45]:
statsmodels.stats.weightstats.ttest_ind(treat.sp_total_days, cont.sp_total_days,alternative="two-sided",usevar="unequal")

(9.763856585869124, 2.240495943817261e-22, 6944.90477379652)

In [56]:
welch_ttest(treat.sp_total_days, cont.sp_total_days,'unequal')

,T statistic,df,pvalue 2 sided,Difference in mean,lb,ub
0,10.664961,5605.787279,2.649640e-26,155.7965,127.158657,184.434343


In [57]:
np.mean(treat.sp_total_days)

251.37575

In [58]:
np.mean(cont.sp_total_days)

95.57925

# Polk county + Drug

In [12]:
df_drug_polk =sp_county_crime_type(county='polk',crime_type='drug',df= raw_clean)

In [13]:
df_drug_polk.head()

,Unnamed: 0,fstnm,lstnm,age,dob,race,gender,county,totpts,judge,...,cjimp,spimp,primp,lsviol,rpviol,faviol,fips,state,pop_2010,sp_cj_total_days
215,215,LEVI,JOHNSON,25,1981-09-08,WHITE,MALE,polk,83.1,DURRANCE,...,NO,YES,NO,0,0,0,12105,FL,602095.0,1440
471,471,KAREN,LOONEY,42,1971-07-08,WHITE,FEMALE,polk,46.9,"OJEDA,REINALDO",...,NO,YES,NO,0,0,0,12105,FL,602095.0,426
798,798,CHARLIE,ATKINS,27,1979-03-19,BLACK,MALE,polk,42.8,MCDONALD,...,NO,YES,NO,0,0,0,12105,FL,602095.0,366
839,839,RODNEY,AMOS,43,1961-01-01,WHITE,MALE,polk,34.6,MCDONALD,...,YES,NO,NO,0,6,0,12105,FL,602095.0,240
1488,1488,CALVIN,THOMPSON,28,1983-09-15,BLACK,MALE,polk,91.8,MCDONALD,...,YES,NO,NO,0,0,0,12105,FL,602095.0,180


In [14]:
df_drug_polk_model = feat_prep(df_drug_polk)

In [15]:
df_drug_polk_model.shape

(15166, 51)

In [16]:
X_continous = df_drug_polk_model.loc[:,['totpts','age']]
X_cont_stand =StandardScaler().fit_transform(X_continous)
df_drug_polk_model['totpts_stand'] = X_cont_stand[:,0]
df_drug_polk_model['age_stand'] = X_cont_stand[:,1]

X = df_drug_polk_model.loc[:,['totpts_stand','age_stand','gender']]

In [17]:
distance_matrix = gower.gower_matrix(X)

In [18]:
# Configuring the parameters of the clustering algorithm
dbscan_cluster = DBSCAN(eps=0.3, 
                        min_samples=2, 
                        metric="precomputed")


dbscan_cluster.fit(distance_matrix)



DBSCAN(eps=0.3, metric='precomputed', min_samples=2)

In [19]:
df_drug_polk_model["cluster"] = dbscan_cluster.labels_

In [20]:
df_drug_polk_model["cluster"].value_counts()

0    11085
1     4081
Name: cluster, dtype: int64

In [21]:
df_drug_polk_model[(df_drug_polk_model["cluster"]==1)&(df_drug_polk_model["race"]=='WHITE')].shape

(3348, 54)

In [22]:
treat_1 = df_drug_polk_model[(df_drug_polk_model["cluster"]==1)&(df_drug_polk_model["race"]=='BLACK')].sample(500)
treat_0 = df_drug_polk_model[(df_drug_polk_model["cluster"]==0)&(df_drug_polk_model["race"]=='BLACK')].sample(2000)

treat= treat_0.append(treat_1)

cont_1 = df_drug_polk_model[(df_drug_polk_model["cluster"]==1)&(df_drug_polk_model["race"]=='WHITE')].sample(500)
cont_0 = df_drug_polk_model[(df_drug_polk_model["cluster"]==0)&(df_drug_polk_model["race"]=='WHITE')].sample(2000)

cont= cont_0.append(cont_1)

In [23]:
welch_ttest(treat.sp_total_days, cont.sp_total_days,'unequal')

,T statistic,df,pvalue 2 sided,Difference in mean,lb,ub
0,4.049354,4968.628632,0.000052,72.6932,37.499743,107.886657


In [24]:
np.mean(treat.sp_total_days)

304.7656

In [25]:
np.mean(cont.sp_total_days)

232.0724

# Pinellas County + Rubbery

In [26]:
df_pinellas_rubbery =sp_county_crime_type(county='pinellas',crime_type='rubbery',df= raw_clean)

In [27]:
df_pinellas_rubbery.shape

(114980, 44)

In [28]:
df_pinellas_rubbery_model = feat_prep(df_pinellas_rubbery)

In [29]:
X_continous = df_pinellas_rubbery_model.loc[:,['totpts','age']]
X_cont_stand =StandardScaler().fit_transform(X_continous)
df_pinellas_rubbery_model['totpts_stand'] = X_cont_stand[:,0]
df_pinellas_rubbery_model['age_stand'] = X_cont_stand[:,1]

#X = df_pinellas_rubbery_model.loc[:,['totpts_stand','age_stand','gender']]

In [43]:
df_pinellas_rubbery_model_sample = df_pinellas_rubbery_model.sample(20000)
X = df_pinellas_rubbery_model_sample.loc[:,['totpts_stand','age_stand','gender']]

In [44]:
distance_matrix = gower.gower_matrix(X)

In [45]:
dbscan_cluster = DBSCAN(eps=0.3, 
                        min_samples=2, 
                        metric="precomputed")


dbscan_cluster.fit(distance_matrix)

DBSCAN(eps=0.3, metric='precomputed', min_samples=2)

In [46]:
df_pinellas_rubbery_model_sample["cluster"] = dbscan_cluster.labels_

In [47]:
df_pinellas_rubbery_model_sample["cluster"].value_counts()

1    14976
0     5024
Name: cluster, dtype: int64

In [50]:
treat_1 = df_pinellas_rubbery_model_sample[(df_pinellas_rubbery_model_sample["cluster"]==1)&(df_pinellas_rubbery_model_sample["race"]=='BLACK')].sample(2000)
treat_0 = df_pinellas_rubbery_model_sample[(df_pinellas_rubbery_model_sample["cluster"]==0)&(df_pinellas_rubbery_model_sample["race"]=='BLACK')].sample(1000)

treat= treat_0.append(treat_1)

cont_1 = df_pinellas_rubbery_model_sample[(df_pinellas_rubbery_model_sample["cluster"]==1)&(df_pinellas_rubbery_model_sample["race"]=='WHITE')].sample(2000)
cont_0 = df_pinellas_rubbery_model_sample[(df_pinellas_rubbery_model_sample["cluster"]==0)&(df_pinellas_rubbery_model_sample["race"]=='WHITE')].sample(1000)

cont= cont_0.append(cont_1)

In [51]:
welch_ttest(treat.sp_total_days, cont.sp_total_days,'unequal')

,T statistic,df,pvalue 2 sided,Difference in mean,lb,ub
0,4.540903,5761.081102,0.000006,128.498667,73.02387,183.973463


In [52]:
np.mean(treat.sp_total_days)

380.43733333333336

In [53]:
np.mean(cont.sp_total_days)

251.93866666666668